## Importing libraries

In [1]:
import random

import numpy as np
import pandas as pd
import torch
from scipy.special import softmax
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    mean_absolute_error,
)
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping, LRScheduler
from skorch.dataset import ValidSplit
from torch import cuda, nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
from torchvision.transforms import Compose, ToTensor

from dlordinal.datasets import FGNet
from dlordinal.losses import (
    BetaLoss,
    BinomialLoss,
    CORNLoss,
    EMDLoss,
    GeometricLoss,
    TriangularLoss,
    WKLoss,
)
from dlordinal.metrics import accuracy_off1, amae, mmae, ranked_probability_score
from dlordinal.output_layers import COPOC
from dlordinal.wrappers import CORNClassifierWrapper

## Dataset
Download `FGNet` dataset.

In [2]:
fgnet_train = FGNet(
    root="./datasets",
    download=True,
    train=True,
    transform=Compose([ToTensor()]),
)

fgnet_test = FGNet(
    root="./datasets",
    download=True,
    train=False,
    transform=Compose([ToTensor()]),
)

num_classes = len(fgnet_train.classes)
classes = fgnet_train.classes
targets = fgnet_train.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device


## Metrics 
Metrics to evaluate different ordinal losses.

In [3]:
def is_unimodal(probs):
    """Check if a 1D array is unimodal (increases to a peak, then decreases)."""
    peak_idx = np.argmax(probs)
    # Increasing up to peak
    inc = np.all(np.diff(probs[: peak_idx + 1]) >= 0)
    # Decreasing after peak
    dec = np.all(np.diff(probs[peak_idx:]) <= 0)
    return inc and dec


def check_unimodality(y_pred):
    """Check unimodality for each row in y_pred and return the proportion."""
    unimodal_flags = np.array([is_unimodal(row) for row in y_pred])
    # Proportion of rows that are unimodal
    proportion = np.mean(unimodal_flags)
    print(
        f"Unimodal predictions: {np.sum(unimodal_flags)} / {len(y_pred)} ({proportion})"
    )
    return proportion


def calculate_metrics(y_true, y_pred):

    if np.allclose(np.sum(y_pred, axis=1), 1):
        y_pred_proba = y_pred
    else:
        y_pred_proba = softmax(y_pred, axis=1)

    y_pred_max = np.argmax(y_pred, axis=1)

    # Metrics
    amae_metric = amae(y_true, y_pred)
    mmae_metric = mmae(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred_max)
    acc = accuracy_score(y_true, y_pred_max)
    acc_1off = accuracy_off1(y_true, y_pred)
    qwk = cohen_kappa_score(y_true, y_pred_max, weights="quadratic")
    rps = ranked_probability_score(y_true, y_pred_proba)
    # Check unimodality
    unimodal_prop = check_unimodality(y_pred_proba)

    metrics = {
        "ACC": acc,
        "1OFF": acc_1off,
        "MAE": mae,
        "QWK": qwk,
        "AMAE": amae_metric,
        "MMAE": mmae_metric,
        "RPS": rps,
        "Unimodality": unimodal_prop,
    }

    for key, value in metrics.items():
        print(f"{key}: {value}")

    print(confusion_matrix(y_true, y_pred_max))

    return metrics

## Experiment
We want to do a brief comparison of several (ordinal) losses using `PyTorch` and `Skorch`
with ResNet18, a pre-trained convolutional neural network, as the model architecture. 
Concretely, we compare Cross Entropy (CE) Loss with several ordinal approaches from the dlordinal library:

- Cross Entropy (CE) Loss
- Squared Earth Mover's Distance (EMD) Loss [1]
- Weighted Kappa Loss [2]
- Binomial Cross Entropy Loss  [3]
- Triangular Cross Entropy Loss [4]
- Beta Cross Entropy Loss [5]
- Geometric Cross Entropy Loss [6]
- Conformal prediction sets for ordinal classification (COPOC) [7]

[1] Hou, L., Yu, C. P., & Samaras, D. (2016). Squared earth mover's distance-based loss for training deep neural networks. arXiv preprint arXiv:1611.05916. 

[2] de La Torre, J., Puig, D., & Valls, A. (2018). Weighted kappa loss function for multi-class classification of ordinal data in deep learning. Pattern Recognition Letters, 105, 144-154.

[3] Liu, X., Fan, F., Kong, L., Diao, Z., Xie, W., Lu, J., & You, J. (2020). Unimodal regularized neuron stick-breaking for ordinal classification. Neurocomputing, 388, 34-44.

[4] Vargas, V. M., Gutiérrez, P. A., Barbero-Gómez, J., & Hervás-Martínez, C. (2023). Soft labelling based on triangular distributions for ordinal classification. Information Fusion, 93, 258-267.

[5] Vargas, V. M., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). Unimodal regularisation based on beta distribution for deep ordinal regression. Pattern Recognition, 122, 108310.

[6] Haas, S., & Hüllermeier, E. (2023, September). Rectifying bias in ordinal observational data using unimodal label smoothing. In Joint European Conference on Machine Learning and Knowledge Discovery in Databases (pp. 3-18). Cham: Springer Nature Switzerland.

[7] Dey, P., Merugu, S., & Kaveri, S. R. (2023). Conformal prediction sets for ordinal classification. Advances in Neural Information Processing Systems, 36, 879-899.



In [ ]:
# Loss functions
losses = [
    CORNLoss(num_classes=num_classes).to(device),
    CrossEntropyLoss().to(device),
    COPOC().to(device),
    EMDLoss(num_classes=num_classes).to(device),
    WKLoss(num_classes=num_classes, use_logits=True).to(device),
    BinomialLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    TriangularLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    BetaLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(device),
    GeometricLoss(
        base_loss=nn.CrossEntropyLoss(),
        num_classes=num_classes,
        alphas=[0.15, 0.35, 0.35, 0.35, 0.35, 0.15],
    ).to(device),
]

# Evaluate each loss K times with different seeds to obtain a more robust result
K = 3

result = pd.DataFrame()


def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # Add deterministic settings
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

for loss_fn in losses:

    for k in range(K):
        # Make results reproducible
        seed = k
        set_seed(seed)  # Use K different seeds

        # Initialize ResNet18 model
        model = models.resnet18(weights="IMAGENET1K_V1")
        if type(loss_fn).__name__ == "COPOC":
            model.fc = nn.Sequential(
                nn.Linear(model.fc.in_features, num_classes), loss_fn
            )
            loss = CrossEntropyLoss().to(device)
        elif type(loss_fn).__name__ == "CORNLoss":
            model.fc = nn.Linear(model.fc.in_features, num_classes - 1)
            loss = loss_fn
        else:
            model.fc = nn.Linear(model.fc.in_features, num_classes)
            loss = loss_fn
        model.to(device)

        # Skorch estimator
        estimator = NeuralNetClassifier(
            module=model,
            criterion=loss,
            optimizer=Adam,
            lr=0.001,
            max_epochs=30,
            # verbose=0,
            train_split=ValidSplit(
                0.1, random_state=seed
            ),  # Use 10% of the data for validation
            callbacks=[
                EarlyStopping(patience=5, monitor="valid_loss"),
                LRScheduler(policy=ReduceLROnPlateau, patience=3, factor=0.5),
            ],
            device=device,
            batch_size=200,
        )

        if type(loss_fn).__name__ == "CORNLoss":
            estimator = CORNClassifierWrapper(estimator)

        print("#" + str(k) + " " + type(loss_fn).__name__)

        estimator.fit(
            X=fgnet_train, y=torch.tensor(fgnet_train.targets, dtype=torch.long)
        )

        test_probs = estimator.predict_proba(fgnet_test)

        metrics = calculate_metrics(np.array(fgnet_test.targets), test_probs)
        print("\n")

        df = pd.DataFrame([metrics])
        df["iteration"] = k
        df["loss"] = type(loss_fn).__name__

        result = pd.concat([result, df], ignore_index=True)

#0 CORNLoss
  epoch    train_loss    valid_acc    valid_loss      lr     dur
-------  ------------  -----------  ------------  ------  ------
      1        0.5656       0.0370        0.4164  0.0010  3.6272
      2        0.2506       0.1235        0.4967  0.0010  0.9071
      3        0.1155       0.1111        0.5312  0.0010  0.7896
      4        0.0462       0.0494        0.4913  0.0010  0.7873
      5        0.0182       0.0617        0.5726  0.0010  0.7929
Stopping since valid_loss has not improved in the last 5 epochs.
Unimodal predictions: 201 / 201 (1.0)
ACC: 0.4925373134328358
1OFF: 0.8855721393034826
MAE: 0.6368159203980099
QWK: 0.7405623716204963
AMAE: 0.7086219336219336
MMAE: 1.5
RPS: 0.6368159203980099
Unimodality: 1.0
[[16  4  1  1  0  0]
 [ 5 22 22 11  0  0]
 [ 0  3  9 19  2  0]
 [ 0  0  2 39  1  0]
 [ 0  0  1 17 11  1]
 [ 0  0  2  5  5  2]]


#1 CORNLoss
  epoch    train_loss    valid_acc    valid_loss      lr     dur
-------  ------------  -----------  ------------  -

## Result
Displays the mean results for each loss over the K iterations.

In [5]:
result.set_index(["loss", "iteration"], inplace=True)
result.groupby("loss").agg("mean").style.highlight_max(
    axis=0, subset=["ACC", "1OFF", "QWK"], color="green"
).highlight_min(axis=0, subset=["MAE", "AMAE", "MMAE", "RPS"], color="green")

,ACC,1OFF,MAE,QWK,AMAE,MMAE,RPS,Unimodality
loss,,,,,,,,
BetaLoss,0.550580,0.925373,0.542289,0.793862,0.650337,1.500000,0.401889,0.865672
BinomialLoss,0.510779,0.917081,0.580431,0.780344,0.678692,1.476190,0.448493,1.000000
COPOC,0.398010,0.789386,0.917081,0.623863,1.029377,1.904762,0.675505,1.000000
CORNLoss,0.522388,0.893864,0.615257,0.752736,0.699591,1.500000,0.615257,1.000000
CrossEntropyLoss,0.514096,0.847430,0.693201,0.663807,0.879521,2.023810,0.594642,0.323383
EMDLoss,0.555556,0.910448,0.563847,0.772693,0.660714,1.404762,0.473208,0.439469
GeometricLoss,0.557214,0.905473,0.565506,0.766457,0.674880,1.547619,0.409479,0.936982
TriangularLoss,0.585406,0.912106,0.515755,0.806652,0.623954,1.357143,0.400246,0.769486
WKLoss,0.535655,0.920398,0.555556,0.781703,0.673641,1.571429,0.505862,0.681592
